In [1]:
%matplotlib inline

import os
import json
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm as loadingbar

In [2]:
from unified_image_reader import Image
from dataset import Dataset, LabelManager, label_extractor
from filtration import FilterBlackAndWhite, FilterHSV, FilterFocusMeasure, FilterManager

In [3]:
import cell_level

In [12]:
def extract_cell_features_from_image(img_path, filtration=None, csv_path='features.csv'):
    # get features
    features = cell_level.feature_engineering.FeatureManager(as_vector=True)
    feature_dataframe = pd.DataFrame(columns = ['RegionNum','CellNum']+list(features.get_empty_feature_set().keys()))
    # get model
    model_manager = cell_level.segmentation.ModelManager()
    model = model_manager.load_model("StarDist")
    # set up iterator
    region_generator = None
    if filtration is None:
        img = Image(img_path)
        region_generator = loadingbar(enumerate(img), total=img.number_of_regions())
    else:
        dataset = Dataset(
            data_dir = img_path,
            labels = LabelManager(img_path, label_extractor.LabelExtractorNoLabels()),
            filtration = FilterManager([
                FilterBlackAndWhite(),
                FilterHSV(),
                FilterFocusMeasure()
            ])
        )
        def iterate_over_dataset():
            for i in range(len(dataset)):
                yield i, dataset[i]
        region_generator = loadingbar(iterate_over_dataset(), total=len(dataset))
    # iterate over regions
    for region_num, region in region_generator:
        cells = model.get_instances(region)
        for cell_num, cell_instance in loadingbar(enumerate(cells), leave=False):
            feature_data = features(cell_instance)
            feature_data['RegionNum'] = region_num
            feature_data['CellNum'] = cell_num
            feature_dataframe.append(feature_data, ignore_index=True)
    # save the feature dataframe
    feature_dataframe.to_csv(csv_path)


In [13]:
img_path = "/workspaces/dev-container/testing/training-simple/test-wsi/0/84429T_003.tif"

In [14]:
extract_cell_features_from_image(img_path)

  0%|          | 0/16625 [00:00<?, ?it/s]

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


100%|██████████| 16625/16625 [1:36:24<00:00,  2.87it/s]
